# Order Placement Test - UNH Bull Put Spread with Error 321 Fix

This notebook tests the actual order placement that was failing with Error 321, with enhanced error handling and account specification.

**Test Objective:**
- Place the exact UNH Bull Put Spread order that failed with Error 321
- Add explicit account specification to fix potential validation issue
- Capture detailed error information if Error 321 still occurs
- Test different order configurations to isolate the root cause

**WARNING: This will place REAL orders in paper trading account DU9233079**

In [9]:
# Import all required modules
import sqlite3
import pandas as pd
import datetime
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract, ComboLeg
from ibapi.order import Order
from ibapi.utils import iswrapper
import time
import threading
import logging
import os
import random
import json

# Enhanced logging setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("Modules imported successfully")
print("WARNING: This notebook will place REAL orders in paper trading!")

Modules imported successfully


In [10]:
# Enhanced IBWrapper with detailed error tracking
class EnhancedIBWrapper(EWrapper):
    def __init__(self):
        super().__init__()
        self.next_order_id = None
        self.contract_details = {}
        self.mid_prices = {}
        self.combo_ids = {}
        self.market_status = "unknown"
        self.order_status = {}
        self.error_messages = []
        self.managed_accounts = []

    @iswrapper
    def nextValidId(self, orderId: int):
        self.next_order_id = orderId
        logger.info(f"Next Valid Order ID: {orderId}")

    @iswrapper
    def managedAccounts(self, accountsList: str):
        self.managed_accounts = accountsList.split(',')
        logger.info(f"Managed Accounts: {self.managed_accounts}")
    
    @iswrapper
    def tickPrice(self, reqId, tickType, price, attrib):
        if tickType in (1, 2):  # Bid or Ask
            if reqId not in self.mid_prices:
                self.mid_prices[reqId] = {"bid": None, "ask": None, "last": None, "model": None}
            self.mid_prices[reqId]["bid" if tickType == 1 else "ask"] = price
            logger.info(f"Received {'bid' if tickType == 1 else 'ask'} price for req_id {reqId}: {price}")

    @iswrapper
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", connectionClosed=False):
        error_info = {
            'reqId': reqId,
            'errorCode': errorCode,
            'errorString': errorString,
            'advancedOrderRejectJson': advancedOrderRejectJson,
            'timestamp': datetime.datetime.now().isoformat()
        }
        self.error_messages.append(error_info)
        
        # Special handling for Error 321
        if errorCode == 321:
            logger.error(f"ERROR 321 DETECTED!")
            logger.error(f"   Request ID: {reqId}")
            logger.error(f"   Error String: {errorString}")
            logger.error(f"   Advanced Order Reject: {advancedOrderRejectJson}")
            logger.error(f"   This is the exact error we are trying to fix!")
        elif errorCode in [2104, 2107, 2158]:  # Normal connection messages
            logger.info(f"Connection Info {errorCode}: {errorString}")
        else:
            logger.error(f"Error {reqId}: {errorCode} - {errorString}")
        
        # Check for market closed error codes
        if errorCode in [2104, 2119]:
            self.market_status = "closed"
            logger.warning(f"Market appears to be closed: {errorString}")
        
    @iswrapper
    def contractDetails(self, reqId, contractDetails):
        if reqId in self.combo_ids:
            self.combo_ids[reqId] = {
                "conId": contractDetails.contract.conId,
                "symbol": contractDetails.contract.symbol,
                "strike": contractDetails.contract.strike,
                "right": contractDetails.contract.right,
                "expiry": contractDetails.contract.lastTradeDateOrContractMonth,
                "exchange": contractDetails.contract.exchange,
                "currency": contractDetails.contract.currency
            }
            logger.info(f"Contract Details: {self.combo_ids[reqId]['symbol']} {self.combo_ids[reqId]['expiry']} {self.combo_ids[reqId]['strike']} {self.combo_ids[reqId]['right']}, conId: {self.combo_ids[reqId]['conId']}")

    @iswrapper
    def contractDetailsEnd(self, reqId):
        logger.info(f"Contract details request {reqId} completed")

    @iswrapper
    def openOrder(self, orderId, contract, order, orderState):
        logger.info(f"Order {orderId} opened with status: {orderState.status}")
        self.order_status[orderId] = {
            'status': orderState.status,
            'contract': contract,
            'order': order,
            'orderState': orderState
        }

    @iswrapper
    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        logger.info(f"Order {orderId} status update: {status}, filled: {filled}, remaining: {remaining}")
        if orderId in self.order_status:
            self.order_status[orderId].update({
                'latest_status': status,
                'filled': filled,
                'remaining': remaining,
                'avgFillPrice': avgFillPrice
            })

print("Enhanced IBWrapper class defined")

Enhanced IBWrapper class defined


In [11]:
# Enhanced IBApp with account specification and better error handling
class EnhancedIBApp(EnhancedIBWrapper, EClient):
    def __init__(self):
        EnhancedIBWrapper.__init__(self)
        EClient.__init__(self, wrapper=self)

    def create_option_contract(self, symbol, expiry, strike, right):
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "OPT"
        contract.exchange = "SMART"
        contract.currency = "USD"
        contract.lastTradeDateOrContractMonth = expiry.replace("-", "")
        contract.strike = strike
        contract.right = right
        contract.multiplier = "100"
        
        logger.info(f"Created option contract: {symbol} {contract.lastTradeDateOrContractMonth} {strike} {right}")
        return contract
    
    def create_combo_contract(self, symbol, leg_contracts, contract_ids):
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "BAG"
        contract.exchange = "SMART"
        contract.currency = "USD"
        
        buy_contract, sell_contract = leg_contracts
        buy_conId, sell_conId = contract_ids
        
        # Create legs with detailed logging
        leg1 = ComboLeg()
        leg1.conId = buy_conId
        leg1.ratio = 1
        leg1.action = "BUY"
        leg1.exchange = "SMART"
        
        leg2 = ComboLeg()
        leg2.conId = sell_conId
        leg2.ratio = 1
        leg2.action = "SELL"
        leg2.exchange = "SMART"
        
        contract.comboLegs = [leg1, leg2]
        
        logger.info(f"Created combo contract for {symbol}:")
        logger.info(f"   Leg 1: {leg1.action} conId={leg1.conId} ratio={leg1.ratio} exchange={leg1.exchange}")
        logger.info(f"   Leg 2: {leg2.action} conId={leg2.conId} ratio={leg2.ratio} exchange={leg2.exchange}")
        
        return contract
    
    def create_enhanced_limit_order(self, action, quantity, price, account=None):
        # Ensure price is negative for vertical spreads (credit collection)
        if price > 0:
            price = -abs(price)
        
        # Round price to nearest $0.05 increment
        price_increment = 0.05
        rounded_price = round(price / price_increment) * price_increment
        rounded_price = round(rounded_price, 2)
        
        order = Order()
        order.action = action
        order.orderType = "LMT"
        order.totalQuantity = quantity
        order.lmtPrice = rounded_price
        order.tif = "DAY"
        order.transmit = True
        
        # KEY FIX: Add explicit account specification
        if account:
            order.account = account
            logger.info(f"Account specified in order: {account}")
        else:
            logger.warning(f"No account specified in order - this might cause Error 321!")
        
        # Additional order parameters that might help
        order.outsideRth = False  # Only during regular trading hours
        order.hidden = False
        order.discretionaryAmt = 0
        
        logger.info(f"Enhanced {action} limit order created:")
        logger.info(f"   Quantity: {quantity}")
        logger.info(f"   Price: ${rounded_price:.2f}")
        logger.info(f"   TIF: {order.tif}")
        logger.info(f"   Account: {getattr(order, 'account', 'NOT SPECIFIED')}")
        logger.info(f"   Outside RTH: {order.outsideRth}")
        
        return order
    
    def get_contract_details(self, contract, req_id):
        self.combo_ids[req_id] = None
        logger.info(f"Requesting contract details for req_id {req_id}")
        self.reqContractDetails(req_id, contract)
        
        wait_time = 10  # Increased wait time
        start_time = time.time()
        while self.combo_ids[req_id] is None and time.time() - start_time < wait_time:
            time.sleep(0.1)
            
        result = self.combo_ids.get(req_id)
        if result:
            logger.info(f"Contract details received for req_id {req_id}")
        else:
            logger.error(f"Failed to get contract details for req_id {req_id}")
            
        return result

    def place_order_with_detailed_logging(self, order_id, contract, order):
        logger.info(f"PLACING ORDER {order_id}:")
        logger.info(f"   Contract Type: {contract.secType}")
        logger.info(f"   Symbol: {contract.symbol}")
        logger.info(f"   Exchange: {contract.exchange}")
        logger.info(f"   Currency: {contract.currency}")
        
        if contract.secType == "BAG":
            logger.info(f"   Combo Legs: {len(contract.comboLegs)}")
            for i, leg in enumerate(contract.comboLegs):
                logger.info(f"     Leg {i+1}: {leg.action} conId={leg.conId} ratio={leg.ratio}")
        
        logger.info(f"   Order Action: {order.action}")
        logger.info(f"   Order Type: {order.orderType}")
        logger.info(f"   Quantity: {order.totalQuantity}")
        logger.info(f"   Limit Price: {order.lmtPrice}")
        logger.info(f"   Account: {getattr(order, 'account', 'NOT SET')}")
        logger.info(f"   TIF: {order.tif}")
        
        # Clear any previous errors for this order
        self.error_messages = [msg for msg in self.error_messages if msg['reqId'] != order_id]
        
        # Place the order
        logger.info(f"Sending order to IB Gateway...")
        self.placeOrder(order_id, contract, order)
        
        # Wait a moment for immediate responses
        time.sleep(2)
        
        # Check for errors
        order_errors = [msg for msg in self.error_messages if msg['reqId'] == order_id]
        if order_errors:
            logger.error(f"Errors detected for order {order_id}:")
            for error in order_errors:
                logger.error(f"   Error {error['errorCode']}: {error['errorString']}")
        else:
            logger.info(f"No immediate errors for order {order_id}")
        
        return order_errors

print("Enhanced IBApp class defined")

Enhanced IBApp class defined


In [12]:
# Test Configuration - Exact UNH Bull Put Spread from Error 321 log
ORDER_TEST_CONFIG = {
    'ticker': 'UNH',
    'expiry': '2025-08-22',
    'strategy_type': 'Bull Put',
    'strike_buy': 230.0,   # Lower strike (buy)
    'strike_sell': 240.0,  # Higher strike (sell)
    'estimated_premium': 195.0,  # Slightly lower than market for testing
    'ibkr_host': '127.0.0.1',
    'ibkr_port': 4002,  # IB Gateway paper trading port
    'client_id': 7,  # Different client ID to avoid conflicts
    'account': 'DU9233079',  # Explicit account specification
    'place_real_order': True  # Set to True to place actual order
}

print("Order Test Configuration:")
for key, value in ORDER_TEST_CONFIG.items():
    print(f"   {key}: {value}")

print(f"WARNING: place_real_order = {ORDER_TEST_CONFIG['place_real_order']}")
if ORDER_TEST_CONFIG['place_real_order']:
    print("   This WILL place real orders in paper trading account!")
    print("   Orders will need to be cancelled manually if desired.")
else:
    print("   This will simulate order placement without actually sending orders.")

Order Test Configuration:
   ticker: UNH
   expiry: 2025-08-22
   strategy_type: Bull Put
   strike_buy: 230.0
   strike_sell: 240.0
   estimated_premium: 195.0
   ibkr_host: 127.0.0.1
   ibkr_port: 4002
   client_id: 7
   account: DU9233079
   place_real_order: True
   This WILL place real orders in paper trading account!
   Orders will need to be cancelled manually if desired.


In [ ]:
# Test 1: Connection and Contract Validation
def test_connection_and_contracts(config):
    print("=" * 60)
    print("TEST 1: CONNECTION AND CONTRACT VALIDATION")
    print("=" * 60)
    
    app = EnhancedIBApp()
    
    try:
        # Connect to IB Gateway
        logger.info(f"Connecting to IB Gateway at {config['ibkr_host']}:{config['ibkr_port']}")
        app.connect(config['ibkr_host'], config['ibkr_port'], config['client_id'])
        
        # Start API thread
        api_thread = threading.Thread(target=app.run, daemon=True)
        api_thread.start()
        
        # Wait for connection
        timeout = 10
        start_time = time.time()
        while not app.next_order_id and time.time() - start_time < timeout:
            time.sleep(0.1)
        
        if not app.next_order_id:
            logger.error("Failed to connect to IB Gateway")
            return False, None, None, None
        
        logger.info(f"Connected! Next Order ID: {app.next_order_id}")
        logger.info(f"Managed Accounts: {app.managed_accounts}")
        
        # Validate account
        if config['account'] not in app.managed_accounts:
            logger.error(f"Account {config['account']} not found in managed accounts: {app.managed_accounts}")
            return False, None, None, None
        else:
            logger.info(f"Account {config['account']} validated")
        
        # Create and validate contracts
        logger.info(f"Creating contracts for {config['strategy_type']}...")
        
        sell_contract = app.create_option_contract(config['ticker'], config['expiry'], config['strike_sell'], "P")
        buy_contract = app.create_option_contract(config['ticker'], config['expiry'], config['strike_buy'], "P")
        
        # Get contract details
        logger.info("Requesting contract details...")
        
        req_id_sell = app.next_order_id
        app.next_order_id += 1
        sell_details = app.get_contract_details(sell_contract, req_id_sell)
        
        req_id_buy = app.next_order_id  
        app.next_order_id += 1
        buy_details = app.get_contract_details(buy_contract, req_id_buy)
        
        if not sell_details or not buy_details:
            logger.error("Failed to get contract details")
            return False, None, None, None
        
        logger.info(f"Contract validation successful:")
        logger.info(f"   Sell: {sell_details['symbol']} {sell_details['strike']}P conId={sell_details['conId']}")
        logger.info(f"   Buy:  {buy_details['symbol']} {buy_details['strike']}P conId={buy_details['conId']}")
        
        return True, app, sell_details, buy_details
        
    except Exception as e:
        logger.error(f"Connection test failed: {str(e)}")
        return False, None, None, None

# Run connection test
print("Running connection test...")
connection_success, app, sell_details, buy_details = test_connection_and_contracts(ORDER_TEST_CONFIG)

2025-08-01 07:13:42,569 - INFO - Connecting to IB Gateway at 127.0.0.1:4002


Running connection test...
TEST 1: CONNECTION AND CONTRACT VALIDATION


2025-08-01 07:13:42,676 - INFO - sent startApi
2025-08-01 07:13:42,680 - INFO - REQUEST startApi {}
2025-08-01 07:13:42,682 - INFO - SENDING startApi b'\x00\x00\x00\x0871\x002\x007\x00\x00'
2025-08-01 07:13:42,684 - INFO - ANSWER connectAck {}
2025-08-01 07:13:42,704 - INFO - Managed Accounts: ['DU9233079']
2025-08-01 07:13:42,793 - INFO - Next Valid Order ID: 1
2025-08-01 07:13:42,823 - ERROR - Error -1: 1754032422230 - 2104
2025-08-01 07:13:42,830 - ERROR - Error -1: 1754032422241 - 2106
2025-08-01 07:13:42,831 - ERROR - Error -1: 1754032422246 - 2158
2025-08-01 07:13:42,891 - INFO - Connected! Next Order ID: 1
2025-08-01 07:13:42,893 - INFO - Managed Accounts: ['DU9233079']
2025-08-01 07:13:42,897 - INFO - Account DU9233079 validated
2025-08-01 07:13:42,900 - INFO - Creating contracts for Bull Put...
2025-08-01 07:13:42,905 - INFO - Created option contract: UNH 20250822 240.0 P
2025-08-01 07:13:42,912 - INFO - Created option contract: UNH 20250822 230.0 P
2025-08-01 07:13:42,914 - I

2025-08-01 07:13:45,061 - ERROR - Error 3: 1754032425000 - 399
2025-08-01 07:13:45,143 - INFO - Order 3 opened with status: PreSubmitted
2025-08-01 07:13:45,147 - INFO - Order 3 status update: PreSubmitted, filled: 0, remaining: 1
2025-08-01 07:13:45,166 - INFO - Order 3 opened with status: PreSubmitted
2025-08-01 07:13:45,171 - INFO - Order 3 status update: PreSubmitted, filled: 0, remaining: 1
2025-08-01 07:13:45,277 - ERROR - Error -1: 1754032424699 - 2104
2025-08-01 07:13:50,090 - INFO - Order 3 opened with status: Submitted
2025-08-01 07:13:50,093 - INFO - Order 3 status update: Submitted, filled: 0, remaining: 1


In [14]:
# Test 2: Order Placement with Enhanced Error Handling
def test_order_placement(app, sell_details, buy_details, config):
    if not connection_success:
        print("Skipping order test - connection failed")
        return None, []
    
    print("=" * 60)
    print("TEST 2: ORDER PLACEMENT WITH ERROR 321 FIXES")
    print("=" * 60)
    
    try:
        # Create contracts
        sell_contract = app.create_option_contract(config['ticker'], config['expiry'], config['strike_sell'], "P")
        buy_contract = app.create_option_contract(config['ticker'], config['expiry'], config['strike_buy'], "P")
        
        # Create combo contract - EXACT same logic as original
        logger.info("Creating combo contract...")
        combo_contract = app.create_combo_contract(
            config['ticker'], 
            [buy_contract, sell_contract], 
            [buy_details['conId'], sell_details['conId']]
        )
        
        # Create enhanced order with account specification
        limit_price = -abs(config['estimated_premium'] / 100)  # Convert to negative price
        limit_price = round(limit_price * 20) / 20  # Round to nearest 0.05
        
        logger.info("Creating enhanced order with account specification...")
        order = app.create_enhanced_limit_order(
            action="BUY",
            quantity=1,
            price=limit_price,
            account=config['account']  # KEY FIX: Explicit account
        )
        
        if config['place_real_order']:
            # Place the actual order
            order_id = app.next_order_id
            app.next_order_id += 1
            
            logger.info(f"PLACING REAL ORDER {order_id} (UNH Bull Put Spread)")
            logger.info(f"   This is the EXACT order that caused Error 321!")
            logger.info(f"   With fixes: Account={config['account']}, Enhanced validation")
            
            # Place order with detailed logging
            errors = app.place_order_with_detailed_logging(order_id, combo_contract, order)
            
            # Wait for order status updates
            logger.info("Waiting for order status updates...")
            time.sleep(5)
            
            # Check final status
            if errors:
                logger.error(f"ORDER PLACEMENT FAILED WITH ERRORS:")
                for error in errors:
                    logger.error(f"   Error {error['errorCode']}: {error['errorString']}")
                    if error['errorCode'] == 321:
                        logger.error(f"   ERROR 321 REPRODUCED! This confirms the issue.")
            else:
                logger.info(f"ORDER PLACED SUCCESSFULLY!")
                logger.info(f"   Order ID: {order_id}")
                logger.info(f"   Status: {app.order_status.get(order_id, {}).get('status', 'Unknown')}")
                logger.info(f"   Error 321 has been FIXED!")
            
            return order_id, errors
        else:
            logger.info(f"DRY RUN - Order configuration validated but not placed")
            logger.info(f"   Order would be: BUY 1 UNH Bull Put Spread @ ${limit_price:.2f}")
            logger.info(f"   Account: {config['account']}")
            logger.info(f"   To place real order, set place_real_order=True")
            return None, []
            
    except Exception as e:
        logger.error(f"Order placement test failed: {str(e)}")
        import traceback
        logger.error(f"Exception details: {traceback.format_exc()}")
        return None, [{'errorCode': 'EXCEPTION', 'errorString': str(e)}]

# Run order placement test
if connection_success:
    print("Running order placement test...")
    order_id, placement_errors = test_order_placement(app, sell_details, buy_details, ORDER_TEST_CONFIG)
else:
    print("Skipping order placement - connection test failed")
    order_id, placement_errors = None, []

2025-08-01 07:13:43,380 - INFO - Created option contract: UNH 20250822 240.0 P
2025-08-01 07:13:43,381 - INFO - Created option contract: UNH 20250822 230.0 P
2025-08-01 07:13:43,382 - INFO - Creating combo contract...
2025-08-01 07:13:43,386 - INFO - Created combo contract for UNH:
2025-08-01 07:13:43,388 - INFO -    Leg 1: BUY conId=797608326 ratio=1 exchange=SMART
2025-08-01 07:13:43,389 - INFO -    Leg 2: SELL conId=797608409 ratio=1 exchange=SMART
2025-08-01 07:13:43,389 - INFO - Creating enhanced order with account specification...
2025-08-01 07:13:43,390 - INFO - Account specified in order: DU9233079
2025-08-01 07:13:43,390 - INFO - Enhanced BUY limit order created:
2025-08-01 07:13:43,392 - INFO -    Quantity: 1
2025-08-01 07:13:43,393 - INFO -    Price: $-1.95
2025-08-01 07:13:43,394 - INFO -    TIF: DAY
2025-08-01 07:13:43,395 - INFO -    Account: DU9233079
2025-08-01 07:13:43,397 - INFO -    Outside RTH: False
2025-08-01 07:13:43,397 - INFO - PLACING REAL ORDER 3 (UNH Bull Pu

Running order placement test...
TEST 2: ORDER PLACEMENT WITH ERROR 321 FIXES


2025-08-01 07:13:45,439 - ERROR - Errors detected for order 3:
2025-08-01 07:13:45,441 - ERROR -    Error 1754032425000: 399
2025-08-01 07:13:45,445 - INFO - Waiting for order status updates...
2025-08-01 07:13:50,456 - ERROR - ORDER PLACEMENT FAILED WITH ERRORS:
2025-08-01 07:13:50,457 - ERROR -    Error 1754032425000: 399


In [15]:
# Test Summary and Results
def summarize_test_results(app, order_id, placement_errors, config):
    print("=" * 70)
    print("TEST SUMMARY AND ERROR 321 ANALYSIS")
    print("=" * 70)
    
    # Connection Results
    print(f"CONNECTION TEST:")
    if connection_success:
        print(f"   Successfully connected to IB Gateway")
        print(f"   Account {config['account']} validated")
        print(f"   Contract details retrieved successfully")
    else:
        print(f"   Connection failed")
        return
    
    # Order Placement Results
    print(f"ORDER PLACEMENT TEST:")
    if config['place_real_order']:
        if placement_errors:
            print(f"   Order placement failed with {len(placement_errors)} error(s)")
            
            # Check for Error 321 specifically
            error_321_found = any(err.get('errorCode') == 321 for err in placement_errors)
            if error_321_found:
                print(f"   ERROR 321 REPRODUCED!")
                print(f"   This confirms the issue persists even with account specification")
                
                # Analyze Error 321 details
                for err in placement_errors:
                    if err.get('errorCode') == 321:
                        print(f"ERROR 321 DETAILS:")
                        print(f"   Error String: {err.get('errorString', 'N/A')}")
                        print(f"   Advanced Reject: {err.get('advancedOrderRejectJson', 'N/A')}")
                        
                        # Provide specific recommendations based on error details
                        error_string = err.get('errorString', '').lower()
                        if 'account' in error_string:
                            print(f"   RECOMMENDATION: Account-related validation issue")
                            print(f"      - Verify account permissions for combo orders")
                            print(f"      - Check account settings in IB Gateway")
                        elif 'combo' in error_string or 'bag' in error_string:
                            print(f"   RECOMMENDATION: Combo order structure issue")
                            print(f"      - Try placing legs as separate orders")
                            print(f"      - Verify combo leg parameters")
                        elif 'exchange' in error_string:
                            print(f"   RECOMMENDATION: Exchange routing issue")
                            print(f"      - Try specific exchange instead of SMART")
                            print(f"      - Check exchange availability for options")
                        else:
                            print(f"   RECOMMENDATION: General order validation issue")
                            print(f"      - Review all order parameters")
                            print(f"      - Contact IB support for specific error details")
            else:
                print(f"   Order failed with other errors (not Error 321)")
                for err in placement_errors:
                    print(f"   Error {err.get('errorCode', 'N/A')}: {err.get('errorString', 'N/A')}")
        else:
            print(f"   ORDER PLACED SUCCESSFULLY!")
            print(f"   Error 321 has been FIXED by adding account specification!")
            if order_id:
                print(f"   Order ID: {order_id}")
                print(f"   Current Status: {app.order_status.get(order_id, {}).get('latest_status', 'Unknown')}")
                print(f"   Remember to cancel this order manually if desired")
    else:
        print(f"   Order validation completed (dry run mode)")
        print(f"   Order structure appears correct")
    
    # All errors encountered during test
    if hasattr(app, 'error_messages') and app.error_messages:
        print(f"ALL ERRORS ENCOUNTERED:")
        for i, err in enumerate(app.error_messages, 1):
            print(f"   {i}. Error {err['errorCode']}: {err['errorString']}")
    
    # Next Steps
    print(f"NEXT STEPS:")
    if placement_errors:
        if any(err.get('errorCode') == 321 for err in placement_errors):
            print(f"   1. Analyze Error 321 details above")
            print(f"   2. Test alternative order configurations")
            print(f"   3. Try individual leg orders instead of combo")
            print(f"   4. Contact IB support with specific error details")
        else:
            print(f"   1. Address the specific errors encountered")
            print(f"   2. Retry with corrected parameters")
    else:
        print(f"   1. Error 321 issue resolved!")
        print(f"   2. Update vertical_spread_order.py with account specification")
        print(f"   3. Test with other strategies to confirm fix")
        print(f"   4. Deploy fix to production")

# Generate test summary
if connection_success:
    summarize_test_results(app, order_id, placement_errors, ORDER_TEST_CONFIG)

TEST SUMMARY AND ERROR 321 ANALYSIS
CONNECTION TEST:
   Successfully connected to IB Gateway
   Account DU9233079 validated
   Contract details retrieved successfully
ORDER PLACEMENT TEST:
   Order placement failed with 1 error(s)
   Order failed with other errors (not Error 321)
   Error 1754032425000: 399
ALL ERRORS ENCOUNTERED:
   1. Error 1754032422230: 2104
   2. Error 1754032422241: 2106
   3. Error 1754032422246: 2158
   4. Error 1754032425000: 399
   5. Error 1754032424699: 2104
NEXT STEPS:
   1. Address the specific errors encountered
   2. Retry with corrected parameters


In [16]:
# Cleanup and Disconnection
def cleanup_connection(app):
    if connection_success and app:
        print(f"CLEANUP:")
        print(f"   Disconnecting from IB Gateway...")
        
        try:
            time.sleep(2)  # Allow any pending responses
            app.disconnect()
            logger.info(f"Disconnected from IB Gateway")
        except Exception as e:
            logger.error(f"Error during disconnect: {str(e)}")
    
    print(f"TEST COMPLETE!")
    print(f"   Check the logs above for detailed results")
    print(f"   Any orders placed will remain active until manually cancelled")

# Perform cleanup
if 'app' in locals():
    cleanup_connection(app)
else:
    print("TEST COMPLETE - No connection to clean up")

CLEANUP:
   Disconnecting from IB Gateway...


2025-08-01 07:13:52,513 - INFO - disconnecting
2025-08-01 07:13:52,514 - INFO - ANSWER connectionClosed {}
2025-08-01 07:13:52,516 - INFO - Disconnected from IB Gateway


TEST COMPLETE!
   Check the logs above for detailed results
   Any orders placed will remain active until manually cancelled
